# it's time to try defining entropy


1. function to define ROIs (one per "square")
2. create csv with sequence of rois visitation + time spent per visit 
3. function for - log(p)
4. function for spatial entropy 